# typedLang
cbvLang 노트북에서 정의했던 내용을 가져와서 타입 검사가 용이하도록
Expr 문법을 조금 수정하였다.

$\displaystyle
\tau ::= \mathtt{unit} \mid \mathtt{bool} \mid \mathtt{int} \mid \tau \times \tau \mid \tau + \tau \mid \tau\to\tau$

In [1]:
-- 변수 이름은 문자열로 나타낸다
type Nm = String

-- simple types for Expr
data Ty
  = TVar Nm -- 타입 변수
  | UNIT -- 값이 딱 하나 존재
  | BOOL -- 값이 두 개 존재
  | INT  -- 값이 무한히 많이 존재
  | Ty :* Ty   -- product type (pair) 순서쌍 타입 -- 구조체, 레코드
  | Ty :+ Ty   -- sum type (tagged union) -- varient, 유니온 태그 있는
  | Ty :-> Ty  -- 함수 타입
  deriving (Show, Eq)

infixr 9 :*
infixr 8 :+
infixr 7 :->

In [2]:
-- 장난감 함수형 언어 = 람다식 + ...
data Expr
  = Var Nm                 -- x, y, z, ...
  | Unit                   -- ()
  | B Bool                 -- False, True
  | I Int                  -- 0, -1, 1, -2, 2, -3, 3, ...
  | Pair Expr Expr         -- (e1,e2)
  | Fst Expr               -- fst e
  | Snd Expr               -- snd e
  | InL Expr               -- InL e
  | InR Expr               -- InR e
  | Case Expr Expr Expr    -- 하스켈이라면 대략 다음에 해당 case e of { inL x1 -> e1; inR x2 -> e2 }
  | Lam Nm Expr            -- \x.e
  | Rec Nm Expr            -- rec f e
  | App Expr Expr          -- e1 e2
  | If Expr Expr Expr      -- if e then e1 else e2
  | Let Nm Expr Expr       -- let x=e2 in e
  | Add Expr Expr          -- e1 + e2
  | Mul Expr Expr          -- e1 * e2
  | Neg Expr               -- - e
  | Eq Expr Expr           -- e1 == e2
  | Ne Expr Expr           -- e1 /= e2
  | Lt Expr Expr           -- e1 <  e2
  | Gt Expr Expr           -- e1 >  e2
  | Le Expr Expr           -- e1 <= e2
  | Ge Expr Expr           -- e1 >= e2
  | And Expr Expr          -- e1 && e2
  | Or Expr Expr           -- e1 || e2
  | Not Expr               -- not e
  deriving (Show, Eq)

In [3]:
import IHaskell.Display

dispTy t = Display [html("<code>"++dpTy t++"</code>")]
dispExpr e = Display [html("<code>"++dpExpr e++"</code>")]


dpTy UNIT = "unit"
dpTy BOOL = "bool"
dpTy INT = "int"
dpTy (t1 :-> t2) = dpTyArrL t1++"->"++dpTy t2
dpTy (t1 :+ t2) = dpTySum t1++"+"++dpTyArrL t2
dpTy (t1 :* t2) = dpTyProd t1++"*"++dpTySum t2

dpTyArrL t@(_ :-> _) = paren $ dpTy t
dpTyArrL t = dpTy t

dpTySum t@(_ :-> _) = paren $ dpTy t
dpTySum t@(_ :+ _) = paren $ dpTy t
dpTySum t = dpTy t

dpTyProd t@(_ :-> _) = paren $ dpTy t
dpTyProd t@(_ :+ _) = paren $ dpTy t
dpTyProd t@(_ :* _) = paren $ dpTy t
dpTyProd t = dpTy t

dpExpr (Var x) = x
dpExpr (Lam x e) = "λ" ++ x ++ "." ++ dpExpr e
dpExpr (App e1 e2) = dpt e1 ++ " " ++ dps e2
dpExpr (I n) = show n
dpExpr (B b) = show b
dpExpr Unit = "()"
dpExpr (Pair e1 e2) = "("++dpExpr e1++","++dpExpr e2++")"
dpExpr (Fst e) = "fst" ++ dpe e
dpExpr (Snd e) = "snd" ++ dpe e
dpExpr (InL e) = "inL " ++ dpe e
dpExpr (InR e) = "inR " ++ dpe e
dpExpr (Case e e1 e2) = "case " ++ dpe e ++ " of { inL: "++dpExpr e1++"; inR: "++dpExpr e2++" }"
dpExpr (Add e1 e2) = dpe e1++" + "++dpe e2
dpExpr (Mul e1 e2) = dpe e1++" * "++dpe e2
dpExpr (Neg e) = "-"++dpe e
dpExpr (Eq e1 e2) = dpe e1++" == "++dpe e2
dpExpr (Ne e1 e2) = dpe e1++" /= "++dpe e2
dpExpr (Lt e1 e2) = dpe e1++" < "++dpe e2
dpExpr (Gt e1 e2) = dpe e1++" > "++dpe e2
dpExpr (Le e1 e2) = dpe e1++" <= "++dpe e2
dpExpr (Ge e1 e2) = dpe e1++" >= "++dpe e2
dpExpr (And e1 e2) = dpe e1++" && "++dpe e2
dpExpr (Or e1 e2) = dpe e1++" || "++dpe e2
dpExpr (Not e) = "not "++ dpe e
dpExpr (Let x e2 e) = "let "++x++" = "++dpt e2++" in "++dpExpr e
dpExpr (Rec f e) = "rec "++f++" "++dpExpr e
dpExpr (If e e1 e2) = "if "++dpe e++" then "++dpe e1++" else "++dpe e2

dpt t@(Lam{}) = paren (dpExpr t)
dpt t@(Let{}) = paren (dpExpr t)
dpt t@(If{})  = paren (dpExpr t)
dpt t@(Rec{}) = paren (dpExpr t)
dpt t         = dpExpr t

dps s@(I _)   = dpExpr s
dps s@(B _)   = dpExpr s
dps s@Unit    = dpExpr s
dps s@(Pair{})    = dpExpr s
dps s@(Var{}) = dpExpr s
dps s         = paren (dpExpr s)

dpe = dps


paren s = "(" ++ s ++ ")"

In [4]:
INT :-> INT :-> INT == INT :-> (INT :-> INT)
INT :-> INT :-> INT == (INT :-> INT) :-> INT
INT :* INT :+ INT :* INT == (INT :* INT) :+ (INT :* INT)
INT :-> INT :+ INT :-> INT == INT :-> (INT :+ INT) :-> INT
INT :-> INT :* INT :-> INT == INT :-> (INT :* INT) :-> INT

True

False

True

True

True

In [5]:
dpTy (INT :-> INT :-> INT)
dpTy (INT :-> (INT :-> INT))
dpTy ((INT :-> INT) :-> INT)

"int->int->int"

"int->int->int"

"(int->int)->int"

In [6]:
dpTy $ INT :* INT :+ INT :* INT
dpTy $ INT :* (INT :+ INT) :* INT
dpTy $ INT :-> INT :+ INT :-> INT
dpTy $ INT :-> (INT :+ INT) :-> INT

dpTy $ INT :-> INT :* INT :-> INT
dpTy $ (INT :-> INT) :* INT :-> INT
dpTy $ INT :-> INT :* (INT :-> INT)
dpTy $ (INT :-> INT) :* (INT :-> INT)

dpTy $ INT :* INT :* INT :* INT
dpTy $ INT :+ INT :+ INT :+ INT

"int*int+int*int"

"int*(int+int)*int"

"int->int+int->int"

"int->int+int->int"

"int->int*int->int"

"(int->int)*int->int"

"int->int*(int->int)"

"(int->int)*(int->int)"

"int*int*int*int"

"int+int+int+int"

In [7]:
idExpr = Lam "x" (Var "x")

putStrLn $ dpExpr $ Lam "y" (App idExpr (App idExpr (Var "y")))

dispExpr $ Lam "y" (App idExpr (App idExpr (Var "y")))

λy.(λx.x) ((λx.x) y)

λy.(λx.x) ((λx.x) y)

In [8]:
putStrLn $ dpExpr $ I 3 `Add` I 4
putStrLn $ dpExpr $ Let "id" idExpr (App (Var "id") (I 5))

3 + 4

let id = (λx.x) in id 5

----

$\displaystyle\begin{array}{rcrcl}
\sigma&\in&\textit{Env} &=&
\textit{Nm} \longrightarrow^{\hspace{-2.7ex}\textrm{fin}} \textit{Val} \\
\Gamma&\in&\textit{TEnv} &=&
\textit{Nm} \longrightarrow^{\hspace{-2.7ex}\textrm{fin}} \textit{Ty} \\
v&\in&\textit{Val} &=& \textit{Unit}\;\cup\;\textit{Bool}\;\cup\;\textit{Int}\;\cup\;(\textit{Val} \times \textit{Val})\;\cup\; (\textit{Val} + \textit{Val})\;\cup\;(\textit{Expr}_V \times Env) \\
 &   &\textit{Unit}&=&\{ \pmb{(\,)} \} \\
b&\in&\textit{Bool}&=& \{\textbf{False},\textbf{True}\} \\
n&\in&\textit{Int}&=& \{\ldots,-2,-1,0,1,2,\ldots\} \\
\end{array}$

$\displaystyle\begin{array}{rcl}
\textit{Expr}_V &\subset& \textit{Expr}\\
\textit{Expr}_V &=& \{ \lambda x:\tau.e \mid x\in\textit{Nm},~\tau\in\textit{Ty},~e\in\textit{Expr}\}~\cup~\\
            & &\{ \textbf{rec}~(f:\tau)~(\lambda x:\tau_2.e) \mid f\in\textit{Nm},~x\in\textit{Nm},~\tau,\tau_2\in\textit{Ty},~ e\in\textit{Expr}\}
\end{array}$

# 장난감 함수형 언어의 타입 규칙(typing rules)

$\displaystyle \Gamma \vdash e : \tau$

타입규칙은 위와 같은 형태의 3항 관계를 정의한다.

매우쉽게 타입검사를 하라면 좀더 추가적인 타입 정보를 추가해야 될 곳이 아직은 몇 군데 남아 있다.

$\displaystyle
 \frac{}{\Gamma \vdash x : \Gamma(x)}
$

$~$

$\displaystyle
 \frac{\{x\mapsto\tau_1\}\Gamma \vdash e : \tau_2}{
       \Gamma \vdash \lambda x.e ~ : \tau_1 \to \tau_2}
 \quad
 \frac{\{f\mapsto\tau\}\Gamma \vdash e : \tau}{
       \Gamma \vdash \textbf{rec}\;f\;e ~ : \tau}
$

$~$

$\displaystyle
 \frac{}{\Gamma \vdash \pmb{(\,)} : \texttt{unit}}
 \quad
 \frac{}{\Gamma \vdash n : \texttt{int}}
 \quad
 \frac{}{\Gamma \vdash \textbf{False} : \texttt{bool}}
 \quad
 \frac{}{\Gamma \vdash \textbf{True} : \texttt{bool}}
$

$~$

$\displaystyle
 \frac{\Gamma \vdash e_1:\tau_1 \quad \Gamma \vdash e_2:\tau_2}{
       \Gamma \vdash \pmb{(}e_1\pmb{,}\,e_2\pmb{)} : \tau_1\times\tau_2}
 \quad
 \frac{\Gamma \vdash e : \tau_1\times \tau_2}{
       \Gamma \vdash \textbf{fst}\;e : \tau_1}
 \quad
 \frac{\Gamma \vdash e : \tau_1\times \tau_2}{
       \Gamma \vdash \textbf{snd}\;e : \tau_2}
$

$~$

$\displaystyle
 \frac{\Gamma \vdash e : \tau_1}{
       \Gamma \vdash \textbf{inL}\;e : \tau_1 + \tau_2}
 \quad
 \frac{\Gamma \vdash e : \tau_2}{
       \Gamma \vdash \textbf{inR}\;e : \tau_1 + \tau_2}
$

$~$

$\displaystyle
 \frac{\Gamma \vdash e : \tau_1+\tau_2 \quad
       \{x_1\mapsto\tau_1\}\Gamma \vdash e_1:\tau \quad
       \{x_2\mapsto\tau_2\}\Gamma \vdash e_2:\tau}{
       \textbf{case}\;e\;\textbf{of}\,\pmb{\{}\,\textbf{inL}\,\lambda x_1.e_1\,\pmb{;}\;~\textbf{inR}\,\lambda x_2.e_2\,\pmb{\}} : \tau}
$

$~$

$\displaystyle
 \frac{\Gamma \vdash e_1 : \tau_2\to\tau \quad \Gamma \vdash e_2 : \tau_2}{
       \Gamma \vdash e_1~e_2 : \tau}
 \qquad
 \frac{\{x\mapsto \tau_1\}\Gamma \vdash e_2 : \tau_2}{
       \Gamma \vdash \textbf{let}~x = e_1 ~\textbf{in}~ e_2 : \tau_2}
$

$~$

$\displaystyle
 \frac{\Gamma \vdash e : \texttt{bool} \quad \Gamma \vdash e_1:\tau \quad \Gamma \vdash e_2:\tau}{
       \Gamma \vdash \textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_2 : \tau}
$

```haskell
data Maybe a = Nothing | Just a
```

In [9]:
:type lookup
lookup 'a' [('a',4),('b',5),('c',6)]
lookup 'b' [('a',4),('b',5),('c',6)]
lookup 'z' [('a',4),('b',5),('c',6)]

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

Just 4

Just 5

Nothing

In [16]:
show True
show 1

"True"

"1"

In [17]:
tvars = ["T"++show i | i<-[0..]]

unmerge (x:y:zs) = (x:xs, y:ys) where (xs,ys) = unmerge zs
unmerge [x] = ([x], [])
unmerge []  = ([], [])

unmerge3 (x:y:z:ws) = (x:xs, y:ys, z:zs) where (xs,ys,zs) = unmerge3 ws
unmerge3 [x,y] = ([x],[y],[])
unmerge3 [x] = ([x], [],[])
unmerge3 []  = ([], [],[])

In [19]:
take 10 tvars

(tvars1, tvars2) = unmerge tvars

take 5 tvars1
take 5 tvars2

["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9"]

["T0","T2","T4","T6","T8"]

["T1","T3","T5","T7","T9"]

In [34]:
type TEnv = [(Nm,Ty)]

typeof :: TEnv -> Expr -> [Nm] -> (Maybe Ty, [[(Ty,Ty)]])
typeof tenv (Var x) _ = (lookup x tenv, [])
typeof _    Unit    _ = (Just UNIT, [])
typeof _    (B _)   _ = (Just BOOL, [])
typeof _    (I _)   _ = (Just INT, [])
typeof tenv (Pair e1 e2) tvars =
  case (mt1, mt2) of
    (Just t1, Just t2) -> (Just (t1 :* t2), eqns1++eqns2)
    _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1, tvars2) = unmerge tvars
typeof tenv (Fst e) (tv1:tv2:tvars) =
  case mt of
    Just t -> (Just (TVar tv1), [(t, TVar tv1 :* TVar tv2)]:eqns)
    _      -> (Nothing, [])
  where
    (mt, eqns) = typeof tenv e tvars
typeof tenv (Snd e) (tv1:tv2:tvars) =
  case mt of
    Just t -> (Just (TVar tv2), [(t, TVar tv1 :* TVar tv2)]:eqns)
    _      -> (Nothing, [])
  where
    (mt, eqns) = typeof tenv e tvars
typeof tenv (InL e1) (tv:tvars) =
  case mt1 of
    Just t1 -> (Just (t1 :+ TVar tv), eqns)
    _       -> (Nothing, [])
  where
    (mt1, eqns) = typeof tenv e1 tvars
typeof tenv (InR e2) (tv:tvars) =
  case mt2 of
    Just t2 -> (Just (TVar tv :+ t2), eqns)
    _       -> (Nothing, [])
  where
    (mt2, eqns) = typeof tenv e2 tvars
typeof tenv (Case e (Lam x1 e1)
                    (Lam x2 e2)) (tv1:tv2:tvars) =
  case (mt, mt1, mt2) of
    (Just t, Just t1, Just t2) -> (Just t1, [(t, TVar tv1 :+ TVar tv2)]:[(t1,t2)]:eqns++eqns1++eqns2)
    _                          -> (Nothing, [])
  where
    (mt, eqns) = typeof tenv e tvars0
    (mt1, eqns1) = typeof ((x1,TVar tv1):tenv) e1 tvars1
    (mt2, eqns2) = typeof ((x2,TVar tv2):tenv) e2 tvars2
    (tvars0,tvars1,tvars2) = unmerge3 tvars
typeof tenv (Lam x e) (tv:tvars) =
  case mt2 of
    Just t2 -> (Just (TVar tv :-> t2), eqns)
    _       -> (Nothing, [])
  where
    (mt2, eqns) = typeof ((x,TVar tv):tenv) e tvars
typeof tenv (Rec f (Lam x e)) (tv:tvars) = typeof ((f,TVar tv):tenv) (Lam x e) tvars
typeof tenv (App e1 e2) (tv:tvars) =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just (TVar tv), [(t1, t2 :-> TVar tv)]:eqns1++eqns2)
     _                  -> (Nothing, [])
   where
     (mt1, eqns1) = typeof tenv e1 tvars1 
     (mt2, eqns2) = typeof tenv e2 tvars2
     (tvars1, tvars2) = unmerge tvars
typeof tenv (If e e1 e2) tvars =
  case (mt, mt1, mt2) of
    (Just t, Just t1, Just t2) -> (Just t1, [(t,BOOL)]:[(t1,t2)]:eqns++eqns1++eqns2)
    _                          -> (Nothing, [])
  where
    (mt, eqns) = typeof tenv e tvars0
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars0,tvars1,tvars2) = unmerge3 tvars
typeof tenv (Let x e2 e) tvars = typeof tenv (App (Lam x e) e2) tvars
typeof tenv (Add e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just INT, [(t1, INT)]:[(t2, INT)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Mul e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just INT, [(t1, INT)]:[(t2, INT)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Neg e) tvars =
   case mt of
     Just t -> (Just INT, [(t,INT)]:eqns)
     _      -> (Nothing, [])
   where
     (mt, eqns) = typeof tenv e tvars
typeof tenv (Eq e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just BOOL, [(t1,t2)]:[(t1,UNIT),(t1,INT),(t1,BOOL)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Ne e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just BOOL, [(t1,t2)]:[(t1,UNIT),(t1,INT),(t1,BOOL)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Lt e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just BOOL, [(t1,t2)]:[(t1,UNIT),(t1,INT),(t1,BOOL)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Gt e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just BOOL, [(t1,t2)]:[(t1,UNIT),(t1,INT),(t1,BOOL)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Le e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just BOOL, [(t1,t2)]:[(t1,UNIT),(t1,INT),(t1,BOOL)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Ge e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just BOOL, [(t1,t2)]:[(t1,UNIT),(t1,INT),(t1,BOOL)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (And e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just BOOL, [(t1, BOOL)]:[(t2, BOOL)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Or e1 e2) tvars =
   case (mt1, mt2) of
     (Just t1, Just t2) -> (Just BOOL, [(t1, BOOL)]:[(t2, BOOL)]:eqns1++eqns2)
     _                  -> (Nothing, [])
  where
    (mt1, eqns1) = typeof tenv e1 tvars1
    (mt2, eqns2) = typeof tenv e2 tvars2
    (tvars1,tvars2) = unmerge tvars
typeof tenv (Not e) tvars =
   case mt of
     Just t -> (Just BOOL, [(t,BOOL)]:eqns)
     _      -> (Nothing, [])
   where
     (mt, eqns) = typeof tenv e tvars

In [35]:
typeof [("f", TVar "T"), ("x",INT)]
       (App (Var "f") (App (Var "f") (Var "x"))) tvars

(Just (TVar "T0"),[[(TVar "T",TVar "T2" :-> TVar "T0")],[(TVar "T",INT :-> TVar "T2")]])

In [36]:
typeof []
  (Rec "f" (Lam "x"
     (App (Var "f") (App (Var "f") (Add (Var "x") (I 1))))))
  tvars

(Just (TVar "T1" :-> TVar "T2"),[[(TVar "T0",TVar "T4" :-> TVar "T2")],[(TVar "T0",INT :-> TVar "T4")],[(TVar "T1",INT)],[(INT,INT)]])

In [37]:
mapM_ print [[(TVar "T0",TVar "T4" :-> TVar "T2")],[(TVar "T0",INT :-> TVar "T4")],[(TVar "T1",INT)],[(INT,INT)]]

[(TVar "T0",TVar "T4" :-> TVar "T2")]
[(TVar "T0",INT :-> TVar "T4")]
[(TVar "T1",INT)]
[(INT,INT)]

In [38]:
typeof [] (Not (B True)) tvars
typeof [] (Not (I 22)) tvars
typeof [] (Not (Var "x")) tvars
typeof [("x",BOOL)] (Not (Var "x")) tvars
typeof [("x",INT)] (Not (Var "x")) tvars

(Just BOOL,[[(BOOL,BOOL)]])

(Just BOOL,[[(INT,BOOL)]])

(Nothing,[])

(Just BOOL,[[(BOOL,BOOL)]])

(Just BOOL,[[(INT,BOOL)]])

In [39]:
typeof []
  (Lam "x" (App (Var "x") (Var "x")))
  tvars

(Just (TVar "T0" :-> TVar "T1"),[[(TVar "T0",TVar "T0" :-> TVar "T1")]])